In [1]:
import pandas as pd
import numpy as np

from sklearn.metrics import make_scorer,accuracy_score, precision_score, recall_score,f1_score,confusion_matrix,balanced_accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.calibration import CalibratedClassifierCV as SVCclbr
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.svm import SVC


from sklearn.preprocessing import StandardScaler, QuantileTransformer,PowerTransformer,MinMaxScaler,MaxAbsScaler,RobustScaler

import joblib

In [2]:
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_rows', 3000)
df01=pd.read_csv(r'C:\Users\tekin\Desktop\ML\Tweedy\BsnsCntVctUsnm_D1911.csv',index_col=False)
df01.head(10)

,edInput,editor,usScName,text
0,1,5004,business,By_business state link Chinese hack group use malware steal sms text message high rank military government target accord cybersecurity company
1,1,5004,Inc,By_inc Developing empathy compassion daily habit
2,1,5004,BW,By_bw argentine startup want build better bee
3,1,5004,business,By_business South Korean export fall 11th straight month October global trade gloom persist
4,1,5004,business,By_business Oil set weekly drop rise Saudi output trade woe
5,1,5004,Inc,By_inc Hiring hard way find identify true star employee
6,1,5004,business,By_business Qantas ground three Boeing 737 jet find crack near wing
7,1,5004,HarvardBiz,By_harvardbiz true believer power democratic politic make big important change want play personal role that
8,1,5004,Inc,By_inc New research prove kill make strong really true @mindazetlin
9,1,5004,HarvardBiz,By_harvardbiz female lead company get early funding world would better off


In [3]:
SEED=7
np.random.seed(SEED)
y_vect=np.array(df01.edInput)
text_vect=np.array(df01.text[:].values)

In [4]:
X_train, X_test, y_train, y_test =train_test_split(text_vect,
            y_vect,random_state=SEED,test_size=0.1,stratify=y_vect)

In [5]:

class TweetVect(CountVectorizer):
    def __init__(self,max_features=700,ngram_range=(1,5),tokenizer=None,token_pattern='\S+',min_df=1,lowercase=False,stop_words=None):
        super().__init__(tokenizer=None,token_pattern='\S+',\
                    min_df=1,lowercase=False,stop_words=None,ngram_range=ngram_range,max_features=max_features)
    def fit(self, X, y=None):

        self.fit_transform(X)
        return self

    def fit_transform(self,X,y=None):
         return super().fit_transform(X).toarray()

    def transform(self,X,y=None):
        return super().transform(X).toarray()

lemma_vect =  CountVectorizer(tokenizer=None,token_pattern='\S+',
                             min_df=1,lowercase=False,stop_words=None,ngram_range=(1,5),max_features=4000)
X_train=lemma_vect.fit_transform(X_train_raw)
X_test=lemma_vect.transform(X_test_raw)

In [6]:
def specificity(y_true, y_pred):
    spcfc= confusion_matrix(y_true, y_pred)[0,0]/(confusion_matrix(y_true,y_pred)[0,0] + confusion_matrix(y_true, y_pred)[0,1])
    return spcfc
def tn(y_true, y_pred): return confusion_matrix(y_true, y_pred)[0, 0]
def fp(y_true, y_pred): return confusion_matrix(y_true, y_pred)[0, 1]
def fn(y_true, y_pred): return confusion_matrix(y_true, y_pred)[1, 0]
def tp(y_true, y_pred): return confusion_matrix(y_true, y_pred)[1, 1]



myscoring = {'accuracy' : make_scorer(accuracy_score),
             'balanced_acc': make_scorer(balanced_accuracy_score),
             'specificity' : make_scorer(specificity),  
           'precision' : make_scorer( precision_score),
           'f1_score' : make_scorer(f1_score),
            'tp': make_scorer(tp), 'tn': make_scorer(tn),
            'fp': make_scorer(fp), 'fn': make_scorer(fn)}



In [7]:
negative = len(y_vect[y_vect==0])
positive = len(y_vect[y_vect==1])
print(negative,'\n',positive)
total = negative + positive
weight_for_0 = (1 / negative)*(total)/2.0 
weight_for_1 = (1 / positive)*(total)/2.0
#weight_for_1 = 100

scale_pos_weight = negative/positive
num_round = 2

4664 
 8807


In [8]:
pipe2 = Pipeline([('vect',TweetVect(max_features=1500)),('prep', MinMaxScaler()),
                     ('classifier',SVC(class_weight = 'balanced',C=0.01,gamma='auto', max_iter=3000,probability=True))
                   ])

pipe2.fit(X_train,y_train)

c:\users\tekin\appdata\local\programs\python\python36\lib\site-packages\sklearn\svm\_base.py:231: ConvergenceWarning: Solver terminated early (max_iter=3000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


Pipeline(memory=None,
         steps=[('vect',
                 TweetVect(lowercase=False, max_features=1500, min_df=1,
                           ngram_range=(1, 5), stop_words=None,
                           token_pattern='\\S+', tokenizer=None)),
                ('prep', MinMaxScaler(copy=True, feature_range=(0, 1))),
                ('classifier',
                 SVC(C=0.01, break_ties=False, cache_size=200,
                     class_weight='balanced', coef0=0.0,
                     decision_function_shape='ovr', degree=3, gamma='auto',
                     kernel='rbf', max_iter=3000, probability=True,
                     random_state=None, shrinking=True, tol=0.001,
                     verbose=False))],
         verbose=False)

In [10]:
inp=['By_statistacharts US import 2.66 million light vehicle worth total 52.6 billion #mexico plus another59.4 billion worth part via @commercegov #mexicotariffs']
prb=pipe2.predict_proba(inp)
print(prb[0,1])

0.6426119096836753


In [9]:

#joblib.dump(lemma_vect,r'C:\Users\tekin\Desktop\ML\Tweedy\Bsnsvect.pkl')
joblib.dump(pipe2,r'C:\Users\tekin\Desktop\ML\Tweedy\deploy\BusinessSVCPipe.pkl')


['C:\\Users\\tekin\\Desktop\\ML\\Tweedy\\BusinessSVCPipe.pkl']




clf1 = SVCclbr()
clf2=

clf.fit(X_train, y_train_all,
        eval_set=[(X_train, y_train_all), (X_test, y_test)],
        verbose=True)

evals_result = clf.evals_result()




#{'validation_0': {'logloss': ['0.604835', '0.531479']},
#'validation_1': {'logloss': ['0.41965', '0.17686']}}




In [11]:
help(Calib)

Help on class LinearSVC in module sklearn.svm._classes:

class LinearSVC(sklearn.base.BaseEstimator, sklearn.linear_model._base.LinearClassifierMixin, sklearn.linear_model._base.SparseCoefMixin)
 |  Linear Support Vector Classification.
 |  
 |  Similar to SVC with parameter kernel='linear', but implemented in terms of
 |  liblinear rather than libsvm, so it has more flexibility in the choice of
 |  penalties and loss functions and should scale better to large numbers of
 |  samples.
 |  
 |  This class supports both dense and sparse input and the multiclass support
 |  is handled according to a one-vs-the-rest scheme.
 |  
 |  Read more in the :ref:`User Guide <svm_classification>`.
 |  
 |  Parameters
 |  ----------
 |  penalty : str, 'l1' or 'l2' (default='l2')
 |      Specifies the norm used in the penalization. The 'l2'
 |      penalty is the standard used in SVC. The 'l1' leads to ``coef_``
 |      vectors that are sparse.
 |  
 |  loss : str, 'hinge' or 'squared_hinge' (default=